In [3]:
import numpy as np
from PIL import Image

def PCA2D(samples, row_top):
    '''samples are a list of 2D matrices (images)'''
    # 确保所有图片尺寸相同
    assert all(sample.shape == samples[0].shape for sample in samples), "All images must have the same shape."
    
    size = samples[0].shape
    mean = np.zeros(size)
    
    # 计算所有图片的平均值
    for s in samples:
        mean += s
    mean /= float(len(samples))
    
    # 计算协方差矩阵
    cov_row = np.zeros((size[1], size[1]))
    for s in samples:
        diff = s - mean
        cov_row += np.dot(diff.T, diff)
    cov_row /= (float(len(samples)-1))
    
    # 计算协方差矩阵的特征值和特征向量
    row_eval, row_evec = np.linalg.eig(cov_row)
    
    # 特征值从大到小排序
    sorted_index = np.argsort(row_eval)
    sorted_index_top = sorted_index[-row_top:] 
    # 保留前row_top个特征向量
    X = row_evec[:, sorted_index_top]
    
    return X

In [2]:
def restruct(matrix,X):
    b = []
    for ai in matrix1:
        temp = np.dot(np.dot(ai,m),m.T)
        b.append(temp)
    return b

In [4]:
# the RMSRE of 2DPCA
def calculate_RMSRE(original_matrices, reconstructed_matrices):
    """计算RMSRE，这里简化处理，直接计算重构误差的均方根"""
    total_error = 0
    for original, reconstructed in zip(original_matrices, reconstructed_matrices):
        error_matrix = original - reconstructed
        total_error += np.linalg.norm(error_matrix, 'fro')**2  # Frobenius范数的平方
    rmsre = np.sqrt(total_error / (len(original_matrices)))  # 注意分母是矩阵的数量
    return rmsre

In [ ]:
#导入orl数据集
import cv2
import os
import numpy as np

# ORL数据集的路径，请替换为实际路径
orl_path = 'D:\download\GLRAM复现材料\ORL'

# 存储矩阵的文件夹路径
output_folder = 'D:\download\GLRAM复现材料\ORL_M'

# 确保输出文件夹存在
if not os.path.exists(output_folder):
    os.makedirs(output_folder)

# ORL数据集的结构通常是按人分文件夹，每个文件夹内有10张图片
for person_folder in os.listdir(orl_path):
    person_path = os.path.join(orl_path, person_folder)
    if os.path.isdir(person_path):
        for image_file in os.listdir(person_path):
            if image_file.endswith(".pgm"):  # 确保是PGM文件
                image_path = os.path.join(person_path, image_file)
                # 读取PGM图片
                img = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
                
                # 确保图片正确读取
                if img is not None:
                    # 将矩阵保存为文本文件，这里以numpy数组的形式保存为例
                    # 文件名可以包含图片信息，例如人编号和图片编号
                    matrix_file_path = os.path.join(output_folder, f"{person_folder}_{image_file.split('.')[0]}.npy")
                    np.save(matrix_file_path, img)  # 使用numpy保存为.npy文件
                    print(f"Image {image_file} saved as matrix.")
                else:
                    print(f"Failed to read {image_file}.")

In [5]:
def add_matrix_list(path):
    Matrix_list = []
    for j in range(1,41):
        for i in range(1,11):
            np_path = f"{path}\s{j}_{i}.npy"
            matrix = np.load(np_path)
            Matrix_list.append(matrix)
            print(f"矩阵{i}添加完成")
    print('添加完成')
    return Matrix_list


In [ ]:
matrix1=add_matrix_list(path = 'D:\download\GLRAM 复现材料\ORL_M')

In [7]:
result = []
for i in range (2,22,2):
    m = PCA2D(matrix1, i)
    list = restruct(matrix1,m)
    rmsre = calculate_RMSRE(matrix1, list)
    result.append(rmsre)

In [8]:
x  = np.arange(2.,22.,2)

In [ ]:
import matplotlib.pyplot as plt
plt.plot(x,result)
plt.show

In [10]:
m1 = PCA2D(matrix1, 10)
path = 'D:\download\GLRAM 复现材料\ORL_M\s1_1.npy'
A = np.load(path)
Y = np.dot(A,m1)

In [ ]:
import cv2
v = [x[1] for x in m1]
V = np.mat(v)
res = np.dot(np.dot(matrix1[0], V.T), m1.T[[2]])
res = res*255
res = 255 - res
row_im = cv2.imwrite("A0.png",res)
plt.figure(figsize=(10, 10))
plt.imshow(res, cmap='gray')  # 假设是灰度图像
plt.title('k=1')
plt.axis('off')  
plt.show()